### Populate namespace with imports, functions and globals

In [ ]:
%run boilerplate.ipynb

### Location of PST files

In [ ]:
# Edit as appropriate
CACHED_ENRON_DATA_DIR = Path("../RevisedEDRMv1_Complete")

# Generate the list of files to know how many there are
files = list(CACHED_ENRON_DATA_DIR.glob('**/*.pst'))

### Initialize progress and report widgets

In [ ]:
# Progress bar for number of files processed
progress = widgets.IntProgress(
    value=0,
    min=0,
    max=len(files),
    step=1,
    description='Progress:',
    bar_style='',
    orientation='horizontal'
)

# Container for the report
output = widgets.Output()

### Iterate over the PST files and do work

In [ ]:
handler.clear_logs()

# Overall report
report = {'Files': 0, 'Messages': 0, 'Attachments': 0, 'Size': 0, 'Errors': 0}

# Start displaying results
display(Box(children=[output, progress], layout=report_box_layout))

# Iterate over files
for pst_file in files:
    try:
        # Iterate over messages
        with PffArchive(pst_file) as archive:
            for message in archive.messages():
                try:
                    # Do something with the message...
                    _ = archive.format_message(message)

                    # Update report
                    report['Messages'] += 1
                    report['Attachments'] += message.number_of_attachments

                    # Refresh report widget every 100 messages
                    if not report['Messages'] % 100:
                        update(output, report)

                except Exception as exc:
                    # Log error and move on to the next message
                    report['Errors'] += 1
                    logger.exception(exc)

    except Exception as exc:
        # Log error and move on to the next file
        report['Errors'] += 1
        logger.exception(exc)
    
    # Update progress bar
    progress.value += 1
    
    # Update report
    report['Files'] += 1    
    report['Size'] += pst_file.stat().st_size
    
    # Refresh report widget
    update(output, report)

In [ ]:
# Print out errors, if any 
handler.show_logs()